In [1]:
from bse import BSE
from bse.constants import SEGMENT, STATUS
import pandas as pd

# --- init BSE client (it internally rate-limits/throttles) ---
bse = BSE(download_folder="./bse_cache")  # cache folder can be anywhere

# --- step 1: get list of all active equity securities on BSE ---
print("Fetching list of BSE equity securities...")
securities = bse.listSecurities(
    segment=SEGMENT.EQUITY,   # only equity segment
    status=STATUS.ACTIVE,
          group="A"      # only active stocks
)
# 'securities' is a list[dict]
df_A = pd.DataFrame(securities)



Fetching list of BSE equity securities...


In [2]:

# --- step 1: get list of all active equity securities on BSE ---
print("Fetching list of BSE equity securities...")
securities = bse.listSecurities(
    segment=SEGMENT.EQUITY,   # only equity segment
    status=STATUS.ACTIVE,
          group="B"      # only active stocks
)
# 'securities' is a list[dict]
df_B = pd.DataFrame(securities)



Fetching list of BSE equity securities...


In [3]:
df_bse = pd.concat([df_A,df_B],axis=0)

In [4]:
url = "https://archives.nseindia.com/content/equities/EQUITY_L.csv"

df_nse = pd.read_csv(url)

In [5]:

df_nse = df_nse.copy()
df_nse.columns = df_nse.columns.str.strip()

df_nse_small = df_nse[["SYMBOL", "NAME OF COMPANY", "DATE OF LISTING", "ISIN NUMBER"]].rename(
    columns={
        "SYMBOL": "symbol_nse",
        "NAME OF COMPANY": "name_nse",
        "DATE OF LISTING": "date_of_listing",
        "ISIN NUMBER": "isin_number",
    }
)

# ---------------------------------------------------------
# 3) Clean BSE dataframe
# ---------------------------------------------------------
df_bse = df_bse.copy()
df_bse.columns = df_bse.columns.str.strip()

# Remove mutual fund rows (common patterns in BSE list)
mf_filters = (
    (df_bse["GROUP"] == "M") |
    (df_bse["Segment"].str.contains("MF", case=False, na=False)) |
    (df_bse["Issuer_Name"].str.contains("Mutual Fund", case=False, na=False))
)
df_bse = df_bse[~mf_filters]

# BSE name
df_bse["name_bse"] = df_bse["Issuer_Name"].fillna(df_bse["Scrip_Name"])

# Prepare BSE subset including market cap
df_bse_small = df_bse[["scrip_id", "name_bse", "ISIN_NUMBER", "Mktcap"]].rename(
    columns={
        "scrip_id": "symbol_bse",
        "ISIN_NUMBER": "isin_number",
        "Mktcap": "market_cap"
    }
)

# ---------------------------------------------------------
# 4) Merge NSE + BSE on ISIN
# ---------------------------------------------------------
combined = pd.merge(
    df_nse_small,
    df_bse_small,
    on="isin_number",
    how="outer"
)

combined["market_cap"] = pd.to_numeric(combined["market_cap"], errors="coerce")

# ---------------------------------------------------------
# 5) Build final output
# ---------------------------------------------------------
final_df = pd.DataFrame({
    "symbol": combined["symbol_nse"].fillna(combined["symbol_bse"]),
    "name of company": combined["name_nse"].fillna(combined["name_bse"]),
    "date of listing": combined["date_of_listing"].fillna("Not Available"),
    "isin number": combined["isin_number"],
    "market cap": combined["market_cap"],  # Only BSE has it — OK
})

# Sort and clean
final_df = final_df.sort_values("symbol", na_position="last").reset_index(drop=True)
final_df = final_df[~(final_df["market cap"]<1000)]

print("Total rows:", final_df.shape[0])
final_df.to_csv("data/all_stocks_combined.csv",index=None)

Total rows: 1679
